## Company Location and Sector Retrieval

#### Imports

In [6]:
from urllib import request
from pprint import pprint
from bs4 import BeautifulSoup
import json, sys
import pandas as pd
from tqdm import tqdm, tqdm_notebook
import time
import pickle
import csv
import glob
import os

#### Functions

In [7]:
def return_files(directory,string):
    list_of_images = [
        os.path.basename(x) for x in glob.glob("{}\{}*.*".format(directory,string))
    ]

    return list_of_images

#### Creating a data dictionary (All necessary URLS)

In [6]:
base_url = "https://www.londonstockexchange.com"
page = 1

list_of_urls = []

while True:
    url = "https://www.londonstockexchange.com/exchange/prices-and-markets/stocks/indices/summary/summary-indices-constituents.html?index=ASX&page={}".format(
        page
    )
    req = request.Request(
        url,
        headers={
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36"
        },
    )
    contents = request.urlopen(req)
    soup = BeautifulSoup(contents, "html.parser")

    page_limit = int(
        soup.find(id="pi-colonna1-display").find(class_="floatsx").text.split("of ")[1]
    )
    if page > page_limit:
        break

    table = soup.find_all(class_="table_dati")

    info = table[0].find_all(class_="name")

    for i in info:
        try:
            temp = {}
            temp["Company"] = i.find("a").text
            temp["url"] = i.find("a")["href"]
            list_of_urls.append(temp)

            print(temp)
        except:
            continue

    print("Completed page {} of {}".format(page, page_limit))
    print(url)

    page += 1

    time.sleep(5)

pd.DataFrame.from_dict(list_of_urls).to_csv("links.csv")

{'Company': '3I GRP.', 'url': '/exchange/prices-and-markets/stocks/summary/company-summary/GB00B1YW4409GBGBXSET1.html'}
{'Company': '3I INF. ORD', 'url': '/exchange/prices-and-markets/stocks/summary/company-summary/JE00BF5FX167JEGBXSTMM.html'}
{'Company': '4IMPRINT GRP.', 'url': '/exchange/prices-and-markets/stocks/summary/company-summary/GB0006640972GBGBXSSMM.html'}
{'Company': '888 HLDGS', 'url': '/exchange/prices-and-markets/stocks/summary/company-summary/GI000A0F6407GBGBXSTMM.html'}
{'Company': 'A.B.FOOD', 'url': '/exchange/prices-and-markets/stocks/summary/company-summary/GB0006731235GBGBXSET1.html'}
{'Company': 'AA PLC', 'url': '/exchange/prices-and-markets/stocks/summary/company-summary/GB00BMSKPJ95GBGBXSSMM.html'}
{'Company': 'AB NEW INDIA', 'url': '/exchange/prices-and-markets/stocks/summary/company-summary/GB0006048770GBGBXSSMM.html'}
{'Company': 'ABDN.ASN INC', 'url': '/exchange/prices-and-markets/stocks/summary/company-summary/GB00B0P6J834GBGBXSSMM.html'}
{'Company': 'ABDN.

AttributeError: module 'pandas' has no attribute 'Dataframe'

#### Retrieving Info From Website

In [2]:
list_of_urls = pd.read_csv('links.csv')
list_of_urls.drop(columns='Unnamed: 0', inplace=True)
list_of_urls = list_of_urls.to_dict(orient='records')

In [ ]:
list_of_tables = []
error_urls = []
count = 0
range_of_urls = len(list_of_urls)

for i in list_of_urls:

    if count < 335:
        count += 1
        continue

    company_name = i["Company"]
    company_url = i["url"]

    try:
        url = "https://www.londonstockexchange.com{}".format(company_url)
        req = request.Request(
            url,
            headers={
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36"
            },
        )
        contents = request.urlopen(req)
        soup = BeautifulSoup(contents, "html.parser")
        table = soup.find_all("table")[3]
        df = pd.read_html(str(table), index_col=0)[0]
        df = df.transpose()
        df.to_csv(r"FTSE_Results\{}.csv".format(company_name))
        df = df.to_dict(orient="records")
        df[0]["Company"] = company_name
        list_of_tables.extend(df)
        print("Completed {} of {}".format(count, range_of_urls))
        count += 1

    except Exception as e:
        error_urls.append(url)
        print(e)
        print("Bad Url: {}".format(url))
        count += 1

        continue

    time.sleep(2)

pickling_on = open("ftse_incomplete.pickle","wb")
pickle.dump(list_of_tables, pickling_on)
pickling_on.close()

#### Collating all the results

In [22]:
ftse_incomplete = pickle.load(open("ftse_incomplete.pickle", "rb"))

In [16]:
list_of_files = return_files('FTSE_Results','*')

In [25]:
master_df = []
for i in list_of_files:
    try:
        test = pd.read_csv(r"FTSE_Results\{}".format(i))
        test.drop(columns='Unnamed: 0', inplace=True)
        test = test.to_dict(orient="records")
        test[0]['Company'] = i.split('.csv')[0]
        
    except Exception as e:
        print(e)
        print('File Error from: {}'.format(i))
        continue
    master_df.extend(test)

column_headers = list(pd.DataFrame.from_dict(test))
master_df.extend(ftse_incomplete)


Initializing from file failed
File Error from: NB GLOBAL £.csv
Initializing from file failed
File Error from: PRINCESS PRIV £.csv


In [26]:
column_headers

['Admission date',
 'Company',
 'Company address',
 'Company market cap, £m*',
 'Company website',
 'FTSE ICB sector',
 'FTSE ICB subsector']

In [27]:
master_df = pd.DataFrame.from_dict(master_df)
master_df = master_df[column_headers]

#### Fixing Missing Data

In [20]:
companies_to_check = []

for i in master_df:
    try:
        if i['FTSE index'] is not None:
            companies_to_check.append(i['Company'])
    except:
        continue

In [69]:
faulty_data = []
for i in list_of_urls:
    for j in companies_to_check:
        if i['Company'] == j:
            faulty_data.append(i)
            

In [70]:
list_of_tables = []
error_urls = []
count = 0

for i in faulty_data:

    company_name = i["Company"]
    company_url = i["url"]

    try:
        url = "https://www.londonstockexchange.com{}".format(company_url)
        req = request.Request(
            url,
            headers={
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36"
            },
        )
        contents = request.urlopen(req)
        soup = BeautifulSoup(contents, "html.parser")
        table = soup.find_all("table")[2]
        df = pd.read_html(str(table), index_col=0)[0]
        df = df.transpose()
        df.to_csv(r"FTSE_Results\{}.csv".format(company_name))
        df = df.to_dict(orient="records")
        df[0]["Company"] = company_name
        list_of_tables.extend(df)
        print("Completed {} of {}".format(count, len(faulty_data))
        count += 1
        
    except Exception as e:
        error_urls.append(url)
        print(e)
        print("Bad Url: {}".format(url))
        count += 1

        continue

    time.sleep(2)

Completed 0 of 636
Completed 1 of 636
Completed 2 of 636
Completed 3 of 636
Completed 4 of 636
Completed 5 of 636
Completed 6 of 636
Completed 7 of 636
Completed 8 of 636
Completed 9 of 636
Completed 10 of 636
Completed 11 of 636
Completed 12 of 636
Completed 13 of 636


#### Final DataFrame

In [84]:
master_df.rename(
    index=str,
    columns={
        "Company market cap, £m*": "Market Cap (£m)",
    },
    inplace=True,
)
master_df["Market Cap (£m)"] = pd.to_numeric(master_df["Market Cap (£m)"])
master_df.drop_duplicates(subset=['Company address'], keep="first", inplace=True)

In [85]:
master_df.sort_values(by="Market Cap (£m)", ascending=False).head(10)

,Admission date,Company,Company address,Market Cap (£m),Company website,FTSE ICB sector,FTSE ICB subsector
138,20 Jul 2005,RDS 'A',"Shell Centre, York Road, London, SE1 7NA, Unit...",201099.55,http://www.shell.com,Oil & Gas Producers,Integrated Oil & Gas
576,08 Apr 1991,HSBC HLDGS.UK,"8-16 Canada Square, London, E14 5HQ, United Ki...",130956.46,http://www.hsbc.com,Banks,Banks
393,29 Mar 1954,BP,"20 Canada Square, Canary Wharf, London, E14 5N...",110478.15,http://www.bp.com,Oil & Gas Producers,Integrated Oil & Gas
529,22 May 1972,GLAXOSMITHKLINE,"C8 Gsk House, 980 Great West Road, Brentford, ...",76797.42,http://www.gsk.com,Pharmaceuticals & Biotechnology,Pharmaceuticals
345,01 Jun 1993,ASTRAZENECA,"1 Francis Crick Avenue, Cambridge Biomedical C...",72528.89,http://www.astrazeneca....,Pharmaceuticals & Biotechnology,Pharmaceuticals
462,01 Apr 1952,DIAGEO,"Lakeside Drive, London, NW10 7HQ, United Kingdom",72206.63,http://www.diageo.com,Beverages,Distillers & Vintners
394,29 Jan 1962,BR.AMER.TOB.,"Globe House, 4 Temple Place, London, WC2R 2PG,...",63032.81,http://www.bat.com,Tobacco,Tobacco
155,01 Nov 1973,RIO TINTO,"6 St James's Square, London, SW1Y 4AD, United ...",54554.69,http://www.riotinto.com,Mining,General Mining
275,11 Aug 1939,UNILEVER,"PO Box 68, Unilever House, Blackfriars, London...",49738.51,http://www.unilever.com,Personal Goods,Personal Products
142,23 Oct 2007,RECKITT BEN. GP,"103-105 Bath Road, Slough, SL1 3UH, United Kin...",42457.27,http://www.reckittbenck...,Household Goods & Home Construction,Nondurable Household Products


In [87]:
master_df.groupby(by=["FTSE ICB sector",'FTSE ICB subsector']).sum()

Market Cap (£m)
FTSE ICB sector                   FTSE ICB subsector                                      
Aerospace & Defense               Aerospace                                       25174.82
                                  Defense                                         22561.02
Automobiles & Parts               Auto Parts                                        903.07
                                  Automobiles                                      2580.99
Banks                             Banks                                          256463.73
Beverages                         Distillers & Vintners                           72672.63
                                  Soft Drinks                                     12757.21
Chemicals                         Specialty Chemicals                             18356.03
Construction & Materials          Building Materials & Fixtures                   31167.55
                                  Heavy Construction                               3669.88
Electricity                       Conventional Electricity                        14912.71
Electronic & Electrical Equipment Electrical Components & Equipment                4977.07
                                  Electronic Equipment                             9598.23
Equity Investment Instruments     Equity Investment Instruments                   77865.86
Financial Services                Asset Managers                                  32675.57
                                  Consumer Finance                                 5828.55
                                  Investment Services                             25142.87
                                  Mortgage Finance                                  681.11
                                  Specialty Finance                                4088.44
Fixed Line Telecommunications     Fixed Line Telecommunications                   25167.77
Food & Drug Retailers             Food Retailers & Wholesalers                    42078.10
Food Producers                    Farming & Fishing                                 361.61
                                  Food Products                                   27190.21
Forestry & Paper                  Paper                                            6782.94
Gas, Water & Multiutilities       Gas Distribution                                 7881.64
                                  Multiutilities                                  28825.63
                                  Water                                           13433.01
General Industrials               Containers & Packaging                          13346.94
                                  Diversified Industrials                          8809.34
General Retailers                 Apparel Retailers                               13032.95
...                                                                                    ...
Oil & Gas Producers               Integrated Oil & Gas                           311577.70
Oil Equipment & Services          Oil Equipment & Services                         4968.50
Personal Goods                    Clothing & Accessories                           9247.14
                                  Personal Products                               50553.52
Pharmaceuticals & Biotechnology   Biotechnology                                    5179.50
                                  Pharmaceuticals                                156901.05
Real Estate Investment & Services Real Estate Holding & Development               10767.00
                                  Real Estate Services                             1763.68
Real Estate Investment Trusts     Diversified REITs                               10573.35
                                  Hotel & Lodging REITs                            2301.97
                                  Industrial & Office REITs                       19299.06
                                  Residential REITs                    